<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [4]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [6]:
# use requests.get() method with the provided static_url and headers
# assign the response to a object
import requests

# URL du snapshot de la page Wikipédia
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Headers pour simuler un navigateur et éviter un blocage
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

# Effectuer la requête GET
response = requests.get(static_url, headers=headers)

# Vérifier si la page a été récupérée correctement
if response.status_code == 200:
    print("La page a été récupérée avec succès !")
else:
    print(f"Erreur lors de la récupération de la page, code {response.status_code}")


La page a été récupérée avec succès !


Create a `BeautifulSoup` object from the HTML `response`


In [7]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
from bs4 import BeautifulSoup

# Créer l'objet BeautifulSoup à partir du texte HTML de la page
soup = BeautifulSoup(response.text, "html.parser")

# Vérifier le type pour s'assurer que c'est bien un objet BeautifulSoup
print(type(soup))


<class 'bs4.BeautifulSoup'>


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [8]:
# Use soup.title attribute
# Afficher le titre de la page
print(soup.title)

# Si tu veux juste le texte du titre sans les balises HTML :
print(soup.title.string)


<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>
List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [9]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
# Trouver tous les tableaux sur la page
html_tables = soup.find_all("table")

# Vérifier le nombre de tableaux trouvés
print(f"Nombre de tableaux trouvés : {len(html_tables)}")


Nombre de tableaux trouvés : 25


Starting from the third table is our target table contains the actual launch records.


In [10]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [20]:
# Supposons que le premier tableau "wikitable" est celui des lancements Falcon 9
first_launch_table = html_tables[0]

# Trouver toutes les lignes du tableau
rows = first_launch_table.find_all("tr")

# Chercher la première ligne qui contient des <th>
for tr in rows:
    ths = tr.find_all("th")
    if ths:  # si on a trouvé des <th>
        column_names = [' '.join(th.stripped_strings) for th in ths if th.stripped_strings]
        break  # on prend juste la première ligne d'en-tête

# Vérifier les noms extraits
print("Noms de colonnes :", column_names)




Noms de colonnes : []


In [21]:
# Afficher le début du tableau pour inspection
first_launch_table = html_tables[0]

# Afficher les 3 premières lignes HTML du tableau
rows = first_launch_table.find_all("tr")
for i, tr in enumerate(rows[:3]):
    print(f"--- Ligne {i} ---")
    print(tr.prettify())


--- Ligne 0 ---
<tr>
 <td class="col-break">
  <div class="mw-heading mw-heading3">
   <h3 id="Rocket_configurations">
    Rocket configurations
   </h3>
  </div>
  <div class="chart noresize" style="padding-top:10px;margin-top:1em;max-width:420px;">
   <div style="position:relative;min-height:320px;min-width:420px;max-width:420px;">
    <div style="float:right;position:relative;min-height:240px;min-width:320px;max-width:320px;border-left:1px black solid;border-bottom:1px black solid;">
     <div style="position:absolute;left:3px;top:224px;height:15px;min-width:18px;max-width:18px;color:inherit;background-color:LightSteelBlue;-webkit-print-color-adjust:exact;border:1px solid LightSteelBlue;border-bottom:none;overflow:hidden;" title="[[Falcon 9 v1.0]]: 2">
     </div>
     <div style="position:absolute;left:55px;top:224px;height:15px;min-width:18px;max-width:18px;color:inherit;background-color:LightSteelBlue;-webkit-print-color-adjust:exact;border:1px solid LightSteelBlue;border-bottom:

Check the extracted column names


In [22]:
for i, table in enumerate(html_tables):
    print(f"Table {i} a {len(table.find_all('tr'))} lignes")
    # Afficher la première ligne du tableau pour vérifier si c'est le bon
    first_row = table.find("tr")
    print(first_row.text[:200], "...\n")


Table 0 a 1 lignes


Rocket configurations




























5

10

15

20

25

30



'10

'11

'12

'13

'14

'15

'16

'17

'18

'19

'20

'21




  Falcon 9 v1.0
  Falcon 9 v1.1
  Falcon 9 Full Thrust
  Fal ...

Table 1 a 1 lignes


Launch outcomes

























10

20

30

40

50



'10

'11

'12

'13

'14

'15

'16

'17

'18

'19

'20

'21

'22




  Loss before launch
  Loss during flight
  Partial failure
  Succe ...

Table 2 a 16 lignes

Flight No.

Date andtime (UTC)

Version,Booster [b]

Launch site

Payload[c]

Payload mass

Orbit

Customer

Launchoutcome

Boosterlanding
 ...

Table 3 a 13 lignes

Flight No.

Date andtime (UTC)

Version,Booster[b]

Launch site

Payload[c]

Payload mass

Orbit

Customer

Launchoutcome

Boosterlanding
 ...

Table 4 a 15 lignes

Flight No.

Date andtime (UTC)

Version,Booster[b]

Launch site

Payload[c]

Payload mass

Orbit

Customer

Launchoutcome

Boosterlanding
 ...

Table 5 a 19 lignes

Flight No.

Date andtime (UTC)

V

In [23]:
first_launch_table = html_tables[2]  # tableau des lancements

column_names = []
# Chercher tous les th et td de la première ligne du tableau
first_row = first_launch_table.find("tr")
for cell in first_row.find_all(["th", "td"]):
    col_name = ' '.join(cell.stripped_strings)
    if col_name and not col_name.isdigit():
        column_names.append(col_name)

print("Noms de colonnes extraits :", column_names)


Noms de colonnes extraits : ['Flight No.', 'Date and time ( UTC )', 'Version, Booster [ b ]', 'Launch site', 'Payload [ c ]', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']


In [24]:
print(column_names)

['Flight No.', 'Date and time ( UTC )', 'Version, Booster [ b ]', 'Launch site', 'Payload [ c ]', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [26]:
import pandas as pd

# Colonnes extraites depuis le tableau HTML
column_names = ['Flight No.', 'Date and time ( UTC )', 'Version, Booster [ b ]',
                'Launch site', 'Payload [ c ]', 'Payload mass',
                'Orbit', 'Customer', 'Launch outcome', 'Booster landing']

# Nettoyer les noms de colonnes : retirer parenthèses, crochets et doubles espaces
clean_columns = [col.replace("(", "").replace(")", "")
                    .replace("[", "").replace("]", "")
                    .replace("  ", " ").strip()
                 for col in column_names]

# Initialiser le dictionnaire avec ces colonnes comme clés
launch_dict = {col: [] for col in clean_columns}

# Si tu veux, tu peux supprimer une colonne inutile
# Par exemple "Date and time UTC" si tu veux la scinder en Date et Time
if 'Date and time UTC' in launch_dict:
    del launch_dict['Date and time UTC']

# Ajouter de nouvelles colonnes pour l'analyse
launch_dict['Date'] = []
launch_dict['Time'] = []
launch_dict['Version Booster'] = []

# Vérifier les clés
print("Clés du dictionnaire :", launch_dict.keys())

# Convertir en DataFrame vide (prêt pour remplir les données)
launch_df = pd.DataFrame(launch_dict)
print(launch_df.head())


Clés du dictionnaire : dict_keys(['Flight No.', 'Version, Booster b', 'Launch site', 'Payload c', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing', 'Date', 'Time', 'Version Booster'])
Empty DataFrame
Columns: [Flight No., Version, Booster b, Launch site, Payload c, Payload mass, Orbit, Customer, Launch outcome, Booster landing, Date, Time, Version Booster]
Index: []


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [28]:
import requests
from bs4 import BeautifulSoup
import unicodedata
import pandas as pd

# URL du snapshot Wikipédia
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

# Request HTML
response = requests.get(static_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Helper functions
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    out=''.join([bv for i, bv in enumerate(table_cells.strings) if i % 2 == 0][0:-1])
    return out

def landing_status(table_cells):
    return [i for i in table_cells.strings][0]

def get_mass(table_cells):
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass = mass[0:mass.find("kg")+2] if "kg" in mass else mass
    else:
        new_mass = 0
    return new_mass

# Mapping des colonnes du tableau Wikipédia vers noms clairs
column_map = {
    'Flight No.': 'Flight No.',
    'Date and time ( UTC )': 'Date and Time',
    'Version, Booster [ b ]': 'Version Booster',
    'Launch site': 'Launch site',
    'Payload [ c ]': 'Payload',
    'Payload mass': 'Payload mass',
    'Orbit': 'Orbit',
    'Customer': 'Customer',
    'Launch outcome': 'Launch outcome',
    'Booster landing': 'Booster landing'
}

# Initialiser launch_dict
launch_dict = {v: [] for v in column_map.values()}
launch_dict['Date'] = []
launch_dict['Time'] = []

# Parcourir tous les tableaux "wikitable plainrowheaders collapsible"
extracted_row = 0
for table in soup.find_all("table", {"class": "wikitable plainrowheaders collapsible"}):
    for rows in table.find_all("tr"):
        # Vérifier si la première cellule <th> est un numéro de vol
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()
        else:
            flag = False

        row = rows.find_all("td")
        if flag and len(row) >= 9:  # S'assurer qu'on a toutes les colonnes
            extracted_row += 1

            # Flight Number
            launch_dict['Flight No.'].append(flight_number)

            # Date et Time
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)

            # Version Booster
            bv = booster_version(row[1])
            if not bv and row[1].a:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)

            # Launch Site
            launch_site = row[2].a.string if row[2].a else row[2].text.strip()
            launch_dict['Launch site'].append(launch_site)

            # Payload
            payload = row[3].a.string if row[3].a else row[3].text.strip()
            launch_dict['Payload'].append(payload)

            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            # Orbit
            orbit = row[5].a.string if row[5].a else row[5].text.strip()
            launch_dict['Orbit'].append(orbit)

            # Customer
            customer = row[6].a.string if row[6].a else row[6].text.strip()
            launch_dict['Customer'].append(customer)

            # Launch Outcome
            launch_outcome = list(row[7].strings)[0].strip()
            launch_dict['Launch outcome'].append(launch_outcome)

            # Booster Landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

print(f"Lignes extraites : {extracted_row}")

# Convertir en DataFrame
launch_df = pd.DataFrame(launch_dict)
launch_df.head()



Lignes extraites : 121


ValueError: All arrays must be of the same length

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [29]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
